In [1]:

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
print ('Ran')


Ran


In [3]:
app_train=pd.read_csv("C:\\Users\\dbda\\Desktop\\application_train.csv")
app_test=pd.read_csv("C:\\Users\\dbda\\Desktop\\application_test.csv")

print("Shape of training data: {}".format(app_train.shape))
print("Shape of test data: {}".format(app_test.shape))

Shape of training data: (307511, 122)
Shape of test data: (48744, 121)


In [4]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
train_labels = app_train['TARGET']
train_sk_id_curr = app_train['SK_ID_CURR']
test_sk_id_curr = app_test['SK_ID_CURR']

app_train.drop('SK_ID_CURR', inplace=True, axis=1)
app_test.drop('SK_ID_CURR', inplace=True, axis=1)

app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 120)
Testing Features shape:  (48744, 120)


In [6]:
cat_features = [f for f in app_train.columns if app_train[f].dtype == 'object']

In [7]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, query_cols, sorter=sidx)]



In [8]:

cat_features_inds = column_index(app_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_inds)

Cat features are: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
[ 0  1  2  3  9 10 11 12 13 26 30 38 84 85 87 88]


In [13]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB


In [15]:

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in cat_features:
    app_train[col] = le.fit_transform(app_train[col].astype(str))
    app_test[col] = le.fit_transform(app_test[col].astype(str))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [16]:

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(app_train)
app_train = imputer.transform(app_train)
app_test = imputer.transform(app_test)

In [11]:
app_train

array([[0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 2., 0., 1.]])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(app_train, train_labels, 
                                                    test_size = 0.3, 
                                        
                                                    stratify=train_labels)

In [18]:
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
predictions=gaussian.predict(app_test)

In [24]:
from sklearn.metrics import roc_curve, roc_auc_score


print('AUC:', roc_auc_score(y_test, gaussian.predict_proba(X_test)[:,1]))
y_pred = gaussian.predict(X_test)

print('Accuracy score ',accuracy_score(y_test, y_pred))


AUC: 0.6102075895125065
Accuracy score  0.9192663732737876


In [26]:
submission = pd.DataFrame({'SK_ID_CURR': test_sk_id_curr, 'TARGET': predictions})

In [29]:
submission.count()

SK_ID_CURR    48744
TARGET        48744
dtype: int64

In [28]:
submission.to_csv('C:\\Users\\dbda\\Desktop\\result\\baseline_gaussian_imputed_median.csv', index = False)
